In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [10]:
data_train, meta = scipy.io.arff.loadarff('yeast-train.arff')
df_train = pd.DataFrame(data_train)
data_test, meta = scipy.io.arff.loadarff('yeast-test.arff')
df_test = pd.DataFrame(data_test)
df_test.head()

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
0,0.004168,-0.170975,-0.156748,-0.142151,0.058781,0.026851,0.197719,0.041850,0.066938,-0.056617,...,b'0',b'0',b'1',b'1',b'0',b'0',b'0',b'1',b'1',b'0'
1,-0.103956,0.011879,-0.098986,-0.054501,-0.007970,0.049113,-0.030580,-0.077933,-0.080529,-0.016267,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
2,0.509949,0.401709,0.293799,0.087714,0.011686,-0.006411,-0.006255,0.013646,-0.040666,-0.024447,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'1',b'0'
3,0.119092,0.004412,-0.002262,0.072254,0.044512,-0.051467,0.074686,-0.007670,0.079438,0.062184,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
4,0.042037,0.007054,-0.069483,0.081015,-0.048207,0.089446,-0.004947,0.064456,-0.133387,0.068878,...,b'1',b'1',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'


In [11]:
X_train = df_train.iloc[:,:-14].values
y_train = df_train.iloc[:,-14:].values.astype(int)
X_test = df_test.iloc[:,:-14].values
y_test = df_test.iloc[:,-14:].values.astype(int)

In [12]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (1500, 103)
Train_Y:  (1500, 14)
Test_X:  (917, 103)
Test_Y:  (917, 14)


## Accuracy Function

In [5]:
def accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

## MODEL 1: Chain Classifier of LR

In [32]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss, log_loss, f1_score

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(classifier= SVC(kernel='linear'))

# Training logistic regression model on train data
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

In [33]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions.toarray()))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions.toarray()))
print("F1 score = ",f1_score(y_test,predictions.toarray(),average='macro'))

Accuracy =  0.484854857974
Hamming loss =  0.21101417666303163
Log loss =  69.9914612089
F1 score =  0.342504441452


## Model 2: Multi-lable Lazy Learning

In [34]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=10)
# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(X_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(X_test).toarray()
# train
classifier_new.fit(X_train, y_train)
# predict
predictions = classifier_new.predict(X_test)

In [35]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions.toarray()))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions.toarray()))
print("F1 score = ",f1_score(y_test,predictions.toarray(),average='macro'))

Accuracy =  0.478634294338
Hamming loss =  0.20922262034584826
Log loss =  68.2292188009
F1 score =  0.39746128916


## Model 3: Classic ANN

In [13]:
import os
os.environ['CUDA_VISIBLE_DIVICES'] = '-1'
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.backend.tensorflow_backend import clear_session
clear_session()
model = Sequential()
model.add(Dense(512,activation='relu',input_shape=(X_train.shape[1],),kernel_initializer='glorot_uniform'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dropout(0.4))
model.add(Dense(y_train.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform'))
model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               53248     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 14)                3598      
Total params: 188,174
Trainable params: 188,174
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x=X_train,y=y_train,epochs=5,batch_size=50)

Epoch 1/5
1500/1500 [==============================] - 15s 10ms/step - loss: 0.4855 - acc: 0.7747
Epoch 2/5
1500/1500 [==============================] - 0s 98us/step - loss: 0.4415 - acc: 0.7981
Epoch 3/5
1500/1500 [==============================] - 0s 106us/step - loss: 0.4275 - acc: 0.8049
Epoch 4/5
1500/1500 [==============================] - 0s 124us/step - loss: 0.4164 - acc: 0.8107
Epoch 5/5
1500/1500 [==============================] - 0s 134us/step - loss: 0.4080 - acc: 0.8144


In [41]:
from sklearn.metrics import accuracy_score

In [42]:
#Predictions
predictions = model.predict(X_test)
predictions = np.array([list(np.round(x).astype(int)) for x in predictions])
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions))
print("F1 score = ",f1_score(y_test,predictions,average='macro'))

Accuracy =  0.176663031625
Hamming loss =  0.19691540738432778
Log loss =  66.7587095973
F1 score =  0.347239299698
